<h3 style="text-align:center">Mohsen Shiri-Ali Chalaki</h3>
<h5 style="text-align:center">Recognizing music style</h5>

<h5 style="text-align:center">Feature Extraction</h5>

<h3 style="direction:rtl">1- افزودن کتابخانه های موردنیاز</h3>

In [44]:
import librosa
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

   <h3 style="direction:rtl">2- بخش تعریف توابع موردنیاز</h3>

In [ ]:
def extract_tempo(y, sr):
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)  # ضربان موسیقی
    return tempo

In [45]:
# استخراج ویژگی‌های مرتبط با صدای بیس
def extract_bass_features(y, sr):
    # تبدیل فوریه سریع (FFT)
    fft = np.fft.fft(y)
    # دامنه فرکانسی
    freq = np.fft.fftfreq(len(y), 1/sr)

    # مقدار مطلق طیف فرکانسی
    abs_fft = np.abs(fft)

    # انتخاب برخی از بینهایت فرکانس‌های FFT برای باس
    low_freq_mask = (freq >= 20) & (freq <= 200)  # مثلاً از 20 تا 200 هرتز

    # محاسبه قدرت باس
    bass_power = np.sum(abs_fft[low_freq_mask])

    return bass_power

In [46]:
# استخراج ویژگی‌های مرتبط با ترکیبات هارمونیکی
def extract_harmonic_features(y, sr):
    # استخراج طیف هارمونیکی
    harmonic = librosa.effects.harmonic(y)
    # استخراج ویژگی‌های مرتبط با ترکیبات هارمونیکی از طیف هارمونیکی
    harmonic_features = librosa.feature.chroma_cqt(y=harmonic, sr=sr)
    return harmonic_features

In [47]:
# استخراج ویژگی‌های مرتبط با پترن‌های زمانی
def extract_temporal_features(y, sr):
    # استخراج تغییرات شدت صوت در طول زمان
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    # استخراج ویژگی‌های مرتبط با پترن‌های زمانی از تغییرات شدت صوت
    temporal_features = librosa.feature.tempogram(onset_envelope=onset_env, sr=sr)
    return temporal_features

In [ ]:
def extract_rms(y):
    rms = librosa.feature.rms(y=y)
    return rms

<h3 style="direction:rtl">3- تعریف متغیر های موردنیاز</h3>

In [48]:
id=[]
df=pd.DataFrame()

file_Names=[]
tempo_Numbers=[]
rms_Max=[]
rms_Min=[]
rms_Mean=[]
rms_Median=[]
spectrogram_Max=[]
#spectrogram_Min=[]
spectrogram_Mean=[]
mfccs_Max=[]
mfccs_Min=[]
mfccs_Mean=[]
#chromagram_Max=[]
chromagram_Min=[]
chromagram_Mean=[]
countStrongNote=[]
countWeakNote=[]
boss_power=[]
mean_timepattern=[]
label=[]

<h3 style="direction:rtl">3- بخش استخراج ویژگی از فایل های موسیقی</h3>

In [49]:

for dirname, _, filenames in os.walk('./trainMusic'):
    for filename in filenames:

        file_path = os.path.join(dirname, filename)
        id.append(file_path.replace("./trainMusic\\",""))
        y, sr = librosa.load(file_path)
        
        
        tempo=extract_tempo(y=y,sr=sr)
        rms = extract_rms(y=y)
        spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr)
        chromagram = librosa.feature.chroma_stft(y=y, sr=sr)
        

        tempo_Numbers.append(tempo)
        
        rms_Max.append(np.max(rms))
        rms_Min.append(np.min(rms))
        rms_Mean.append(np.mean(rms))
        rms_Median.append(np.median(rms))

        spectrogram_Max.append(np.max(spectrogram))
        spectrogram_Mean.append(np.mean(spectrogram))
        #spectrogram_Min.append(np.min(spectrogram))
    

        mfccs_Max.append(np.max(mfccs))
        mfccs_Min.append(np.min(mfccs))
        mfccs_Mean.append(np.mean(mfccs))
    
        chromagram_Min.append(np.min(chromagram))
        chromagram_Mean.append(np.mean(chromagram))
        #chromagram_Max.append(np.max(chromagram))

        harmonic=extract_harmonic_features(y, sr)
        countStrongNote.append(np.sum((harmonic >= 0.5) & (harmonic <= 0.7)))
        countWeakNote.append(np.sum((harmonic >= 0.2) & (harmonic <= 0.3)))
 
        boss_power.append(int(np.round(extract_bass_features(y, sr))))

        mean_timepattern.append(np.mean(extract_temporal_features(y, sr)))
        

<h3 style="direction:rtl">3- بخش قراردادن ویژگی ها در DataFrame</h3>

In [50]:
id_name=[]
for path in id:
    id_name.append(path.split("\\")[-1])
df['File Name']=id_name
df['Tempo']=tempo_Numbers
df['RMS max']=rms_Max
df['RMS min']=rms_Min
df['RMS mean']=rms_Mean
df['RMS median']=rms_Median
df['Spectrogram max']=spectrogram_Max
#df['Spectrogram min']=spectrogram_Min
df['Spectrogram mean']=spectrogram_Mean
df['mfccs max']=mfccs_Max
df['mfccs min']=mfccs_Min
df['mfccs mean']=mfccs_Mean
#df['chromagram max']=chromagram_Max
df['chromagram min']=chromagram_Min
df['chromagram mean']=chromagram_Mean
df["NumberStrongNote0.5&0.7"]=countStrongNote
df['NumberWeakNote0.2&0.3']=countWeakNote
df['Boss Power']=boss_power
df['Mean TimePattern']=mean_timepattern

labels=[]
for path in df["File Name"]:
   labels.append(path.split(".")[0])
df['Label']=labels

<h3 style="direction:rtl">3- بخش ذخیره ویژگی ها در فایل CSV</h3>

In [52]:
df.to_csv('./MusicDataSet.csv')